In [5]:
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as optim
from functorch import make_functional_with_buffers, vmap, grad, jacrev


In [54]:
class Model(nn.Module):
    def __init__(self, d=2): 
        super(Model, self).__init__()
        self.d = d 
        self.layer1 = nn.Linear(d, d+1, bias=False) 
        # nn.init.zeros_(self.layer1.weight)

        self.layer2 = nn.Linear(d+1, 1, bias=False) 
        # nn.init.zeros_(self.layer2.weight)

    def forward(self,x):
        x = self.layer1(x) 
        x = self.layer2(x)
        return x

In [55]:
device = torch.device('cuda')
model = Model().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr= 0.1)
data = torch.Tensor(np.random.randn(10, model.d)).to(device)

In [56]:
func, params, buffers = make_functional_with_buffers(model)
result = vmap(jacrev(func), (None, None, 0))(params, buffers, data)

g =  data[0][:,None] @ params[1]

In [75]:
for w in result:
    w1 = torch.reshape(w, (10,-1)) 
    print(w1.shape)

res_mat = torch.cat( [w.reshape(10, -1)  for w in result], dim=1)

torch.Size([10, 6])
torch.Size([10, 3])


In [76]:
res_mat.shape

torch.Size([10, 9])

In [59]:
result[0][0]

tensor([[[ 0.1210,  0.1474],
         [ 0.0118,  0.0144],
         [-0.0477, -0.0582]]], device='cuda:0', grad_fn=<SelectBackward0>)

In [60]:
g

tensor([[ 0.1210,  0.0118, -0.0477],
        [ 0.1474,  0.0144, -0.0582]], device='cuda:0', grad_fn=<MmBackward0>)

In [61]:
# init_weights = list(model.parameters())
for p in model.parameters():
    print(p.data.shape)


torch.Size([3, 2])
torch.Size([1, 3])


In [65]:
x = np.random.rand(2,3) 
a_hot = np.zeros(4) 
a_hot[3] = 1 
y = np.kron(a_hot, x) # (da,)

In [66]:
x

array([[0.10470493, 0.14171866, 0.16990051],
       [0.03715569, 0.7648526 , 0.64708154]])

In [3]:
import numpy as np 

A = np.random.randn(3,3,4) 
x = np.random.randn(5,3) 
y = np.dot(x, A) 
print(y.shape)

(5, 3, 4)
